# Data



## Load and Save of Torch model

In [ ]:
import torch
import os 

#device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

from redisai import Client
con = Client(host='evc.re.kr', port=6379)

ag_weights = torch.load("ag_weights.pt", map_location=device)
torch.save(ag_weights, 'ag_weights2.pt')

ag_weights2 = torch.load("ag_weights2.pt", map_location=device)
torch.save(ag_weights2, 'ag_weights3.pt')

ag_weights3 = torch.load("ag_weights3.pt", map_location=device)

In [ ]:
type( ag_weights3 )

In [ ]:
# Torch model
model_path = os.path.join('ag_weights3.pt')
model = open(model_path, 'rb').read()

In [ ]:
type(model)


## [문제있음] Save to Redis

- Redis 캐시에 저장하고 읽어오는 것은 ok
- 읽어온 데이터가 모델로 활용되지 못함


In [23]:
import redis 

RedisHost = 'evc.re.kr'
RedisPort = '20079'
RedisPass = 'redis4evc'

redis_conn = redis.Redis(
    host=RedisHost,
    port=RedisPort,
    #password=RedisPass,
    db=0
)

key   = 'ag_weights'
value = open('ag_weights.pt', 'rb').read()

redis_conn.set(key, value)

True

In [29]:
for name, param in model.named_parameters():
    redis_conn.tensorset(f'{name}',param.data.numpy().cpu().detach())

AttributeError: 'bytes' object has no attribute 'named_parameters'

In [24]:

read_weights = redis_conn.get(key)
torch.save(read_weights, 'ag_weights_from_redis.pt')

In [25]:
#import numpy as np

pipe = redis_conn.pipeline(transaction=False)
pipe = pipe.set('nativeKey', 1)
pipe = pipe.set('ag_weights', model)
pipe.execute()

[True, True]

In [26]:
a = pipe.get('ag_weights')
a

Pipeline<ConnectionPool<Connection<host=evc.re.kr,port=20079,db=0>>>

## just test

In [ ]:

import sys
import io

import redis
import torch
from tqdm.auto import tqdm


ver = sys.version_info
if ver >= (3, 8):
    PICKLE_VERSION = 5
else:
    PICKLE_VERSION = 4

CXN = redis.ConnectionPool(host='192.168.1.6', port=6379, db=0)


class RedisListObject:

    def __init__(self, name):
        self.name = name

    def __len__(self):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            return rdb.llen(self.name)

    def __setitem__(self, index, value):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            if index >= rdb.llen(self.name):
                raise IndexError
            with io.BytesIO() as buf:
                torch.save(value, buf, pickle_protocol=PICKLE_VERSION, _use_new_zipfile_serialization=True)
                if PICKLE_VERSION >= 5:
                    rdb.lset(self.name, index, buf.getbuffer())
                else:
                    rdb.lset(self.name, index, buf.getvalue())

    def __getitem__(self, index):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            if not rdb.exists(self.name):
                raise redis.DataError(f'Dataset named {self.name} does not exist')
            if index >= rdb.llen(self.name):
                raise IndexError
            with io.BytesIO(rdb.lindex(self.name, index)) as buf:
                return torch.load(buf)

    def append(self, value):
        with io.BytesIO() as buf:
            torch.save(value, buf, pickle_protocol=PICKLE_VERSION, _use_new_zipfile_serialization=True)
            #print(len(buf.getvalue()))
            with redis.StrictRedis(connection_pool=CXN) as rdb:
                func = rdb.rpush if rdb.exists(self.name) else rdb.lpush
                if PICKLE_VERSION >= 5:
                    func(self.name, buf.getbuffer())
                else:
                    func(self.name, buf.getvalue())

    def delete(self):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            if rdb.exists(self.name):
                rdb.delete(self.name)
            else:
                raise redis.DataError(f'Dataset named {self.name} does not exist')


class RedisClient:

    def get(self, key):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            if rdb.exists(key):
                return RedisListObject(key)
            else:
                raise redis.DataError(f'Dataset named {key} does not exist')

    def set_data_list(self, key, values):
        try:
            obj = self.get(key)
            obj.delete()
        except:
            obj = RedisListObject(key)

        for item in tqdm(values, desc=f"storing {key}", dynamic_ncols=True):
            obj.append(item)

    def keys(self):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            return rdb.keys()

    def stats(self):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            try:
                return rdb.memory_stats()
            except:
                return rdb.execute_command('MEMORY STATS')

    def check_lens(self, nums):
        try:
            for k, v in nums.items():
                obj = self.get(k)
                if v != 0 and len(obj):
                    return False
        except:
            return False

    def flushdb(self):
        with redis.StrictRedis(connection_pool=CXN) as rdb:
            rdb.flushdb()


if True:
    c = RedisClient()
    print(c.stats())

    data_list = [tuple(torch.rand(10, 10) for _ in range(10)) for _ in range(10)]
    c.set_data_list("test_data_list", data_list)
    print(c.get("test_data_list")[0], c.get("test_data_list")[1])

    #c.flushdb()
    #print(c.stats())

In [ ]:

ag_weights = torch.load("ag_weights.pt", map_location=device)
c.set_data_list("ag_weights", ag_weights)

In [39]:
print(c.get("ag_weights")[0], c.get("ag_weights")[1])
print(c.stats())

features.0.weight features.0.bias
{'peak.allocated': 287071288, 'total.allocated': 1154208, 'startup.allocated': 840976, 'replication.backlog': 0, 'clients.slaves': 0, 'clients.normal': 184552, 'aof.buffer': 0, 'lua.caches': 0, 'db.0': {'overhead.hashtable.main': 152, 'overhead.hashtable.expires': 0}, 'overhead.total': 1025680, 'keys.count': 3, 'keys.bytes-per-key': 104410, 'dataset.bytes': 128528, 'dataset.percentage': '41.032844543457031', 'peak.percentage': '0.40206319093704224', 'allocator.allocated': 1654032, 'allocator.active': 2117632, 'allocator.resident': 8269824, 'allocator-fragmentation.ratio': '1.2802847623825073', 'allocator-fragmentation.bytes': 463600, 'allocator-rss.ratio': '3.9052224159240723', 'allocator-rss.bytes': 6152192, 'rss-overhead.ratio': '11.57256031036377', 'rss-overhead.bytes': 87433216, 'fragmentation': '85.971725463867188', 'fragmentation.bytes': 94589848}


## Pickel 사용 , chat gpt 도움

In [42]:
import torch
import redis
from collections import OrderedDict
import pickle

In [ ]:
# 가상의 모델을 정의하고 학습하는 예제
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = torch.nn.Linear(2, 1)

    def forward(self, x):
        return self.fc(x)

# 모델 인스턴스 생성
model = SimpleModel()

# 모델 학습 (여기에서는 더미 학습)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

for _ in range(100):
    inputs = torch.randn(10, 2)
    labels = torch.randn(10, 1)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

# 학습된 모델을 OrderedDict로 변환
state_dict = model.state_dict()
ordered_dict_model = OrderedDict(state_dict)

# Redis에 모델 저장
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)
redis_client.set('model_A', pickle.dumps(ordered_dict_model))


In [ ]:
# Redis에서 모델 읽어오기
loaded_ordered_dict_model = pickle.loads(redis_client.get('model_A'))

# 빈 모델 인스턴스 생성
loaded_model = SimpleModel()

# OrderedDict를 모델의 state_dict로 로드
loaded_model.load_state_dict(loaded_ordered_dict_model)

# 모델을 사용할 수 있습니다.
input_data = torch.tensor([[1.0, 2.0]])  # 입력 데이터 예제
output = loaded_model(input_data)
print(output)
